## Part 1

In [355]:
TEST_INFILE_1 = "inputs/day_20_test_1.txt"
TEST_INFILE_2 = "inputs/day_20_test_2.txt"
INFILE = "inputs/day_20_1.txt"

#with open(TEST_INFILE_1) as infile:
#with open(TEST_INFILE_2) as infile:
with open(INFILE) as infile:
    lines = infile.read().splitlines()

In [356]:
lines

['%sf -> pz, gj',
 '%zh -> bc, st',
 '%hk -> bc',
 '&bc -> mn, zl, xb, mm, dh, hv, gz',
 '%st -> bc, mm',
 '%gv -> xf, qq',
 '%hv -> xb',
 '%nd -> gj, tr',
 '%zx -> bx, ms',
 '%sc -> ks, gj',
 '%gr -> hn',
 '%pl -> qq, rh',
 '%qc -> sf, gj',
 '%xr -> sc, gj',
 '%zl -> zh',
 '&gj -> ks, ld, sg, xr',
 '%dg -> ll, bx',
 '%nf -> bc, tg',
 '%lz -> cv, qq',
 '%nq -> dg, bx',
 '%rh -> qq, lp',
 '%xf -> qq, qj',
 '%ms -> bx, xh',
 '%mn -> bc, hv',
 '&jm -> rx',
 '%xh -> vt, bx',
 '%pz -> gj',
 '%vq -> bt',
 '%gz -> nf',
 '%bt -> gr',
 '&sg -> jm',
 '%fr -> bx, tb',
 '&lm -> jm',
 '%ld -> cl',
 '%cv -> vq',
 '%cl -> gj, jf',
 '%tr -> gj, sz',
 '%sz -> gj, ld',
 '%dx -> hk, bc',
 '%lr -> bx, fr',
 '%vt -> lr, bx',
 '%ll -> zx',
 'broadcaster -> pl, xr, mn, xc',
 '%lp -> lz',
 '%mm -> gz',
 '&qq -> lm, gr, cv, vq, lp, pl, bt',
 '%xb -> zl',
 '&bx -> ll, xc, db',
 '%tb -> bx',
 '%hn -> gv, qq',
 '%jf -> qc, gj',
 '%qj -> qq',
 '%xc -> bx, pm',
 '%tg -> bc, dx',
 '&dh -> jm',
 '%ks -> nd',
 '&db ->

In [357]:
from collections import defaultdict


class Module:
    def __init__(self, name, module_type, inputs=None, outputs=None):
        self.name = name
        self.module_type = module_type
        self.inputs = inputs if inputs is not None else []
        self.outputs = outputs if outputs is not None else []

    def __repr__(self):
        return f"Module(name={self.name}, module_type={self.module_type}, inputs={self.inputs}, outputs={self.outputs})"
    
    def handle_message(self, message: tuple, message_queue: list, debug=False):
        msg_from, msg_to, msg = message
        if debug: print(f"{msg_from} {'-low' if not msg else '-high'} -> {msg_to}")


class Broadcaster(Module):
    def __init__(self, name, outputs=None):
        super().__init__(name, module_type="broadcaster", inputs=None, outputs=outputs)

    def handle_message(self, message: tuple, message_queue: list, debug=False):
        msg_from, msg_to, msg = message
        if debug: print(f"{msg_from} {'-low' if not msg else '-high'} -> {msg_to}")
        assert msg_from == "button"
        for o in self.outputs:
            message_queue.append((self.name, o, False))


class FlipFlop(Module):
    def __init__(self, name, inputs=None, outputs=None):
        super().__init__(name, module_type="flip_flop", inputs=inputs, outputs=outputs)
        self.state = False

    def handle_message(self, message: tuple, message_queue: list, debug=False):
        msg_from, msg_to, msg = message
        if debug: print(f"{msg_from} {'-low' if not msg else '-high'} -> {msg_to}")
        # if the pulse is low flip, if not do nothing
        if not msg:
            self.state = not self.state
            for o in self.outputs:
                message_queue.append((self.name, o, self.state))


class Conjunction(Module):
    def __init__(self, name, inputs=None, outputs=None):
        super().__init__(name, module_type="conjunction", inputs=inputs, outputs=outputs)
        self.state = {}


    def handle_message(self, message: tuple, message_queue: list, debug=False):
        msg_from, msg_to, msg = message
        if debug: print(f"{msg_from} {'-low' if not msg else '-high'} -> {msg_to}")
        # remember this pulse
        self.state[msg_from] = msg
        # send a low if everyone is high
        if all(self.state.values()):
            for o in self.outputs:
                message_queue.append((self.name, o, False))
        # otherwise send a high
        else:
            for o in self.outputs:
                message_queue.append((self.name, o, True))



In [358]:
modules = {}
# in one pass make sure that we create all our nodes and connect their outputs
for line in lines:
    l, r = line.split(" -> ")
    outs = [o.strip() for o in r.split(",")]
    if l == "broadcaster":
        n = Broadcaster("broadcaster", outputs=outs)
        modules[l] = n
    elif l.startswith("%"):
        l = l.replace("%", "")
        n = FlipFlop(l, outputs=outs)
        modules[l] = n
    elif l.startswith("&"):
        l = l.replace("&", "")
        n = Conjunction(l, outputs=outs)
        modules[l] = n
    else:
        raise ValueError(f"Unknown node type: {l}")

# in a second pass connect up the inputs
for line in lines:
    l, r = line.split(" -> ")
    outs = [o.strip() for o in r.split(",")]
    for out in outs:
        if out in modules:
            modules[out].inputs.append(modules[l.replace("%", "").replace("&", "")])
        else:
            modules[out] = Module(out, "unknown", inputs=[modules[l.replace("%", "").replace("&", "")]], outputs=[])


# in a third pass make sure that all conjunction states are initialized
for module in modules.values():
    if isinstance(module, Conjunction):
        module.state = {i.name: False for i in module.inputs}

In [359]:
modules

{'sf': Module(name=sf, module_type=flip_flop, inputs=[Module(name=qc, module_type=flip_flop, inputs=[Module(name=jf, module_type=flip_flop, inputs=[Module(name=cl, module_type=flip_flop, inputs=[Module(name=ld, module_type=flip_flop, inputs=[Module(name=gj, module_type=conjunction, inputs=[Module(name=sf, module_type=flip_flop, inputs=[...], outputs=['pz', 'gj']), Module(name=nd, module_type=flip_flop, inputs=[Module(name=ks, module_type=flip_flop, inputs=[Module(name=sc, module_type=flip_flop, inputs=[Module(name=xr, module_type=flip_flop, inputs=[Module(name=gj, module_type=conjunction, inputs=[...], outputs=['ks', 'ld', 'sg', 'xr']), Module(name=broadcaster, module_type=broadcaster, inputs=[], outputs=['pl', 'xr', 'mn', 'xc'])], outputs=['sc', 'gj'])], outputs=['ks', 'gj']), Module(name=gj, module_type=conjunction, inputs=[...], outputs=['ks', 'ld', 'sg', 'xr'])], outputs=['nd'])], outputs=['gj', 'tr']), Module(name=sc, module_type=flip_flop, inputs=[Module(name=xr, module_type=flip

In [360]:
def press_button(modules, debug=False):
    messages = [("button", "broadcaster", False)]
    lo, hi = 0, 0
    while len(messages) > 0:
        msg_from, msg_to, msg = messages.pop(0)
        if not msg:
            lo += 1
        else:
            hi += 1
        modules[msg_to].handle_message((msg_from, msg_to, msg), messages, debug)

    return modules, (lo, hi)

In [361]:
total_lo, total_hi = 0, 0
for _ in range(1000):
    modules, (lo, hi) = press_button(modules)
    total_lo += lo
    total_hi += hi

In [350]:
total_lo, total_hi, total_lo * total_hi

(18548, 45383, 841763884)

## Part 2

In [365]:
modules["rx"].inputs

[Module(name=jm, module_type=conjunction, inputs=[Module(name=sg, module_type=conjunction, inputs=[Module(name=gj, module_type=conjunction, inputs=[Module(name=sf, module_type=flip_flop, inputs=[Module(name=qc, module_type=flip_flop, inputs=[Module(name=jf, module_type=flip_flop, inputs=[Module(name=cl, module_type=flip_flop, inputs=[Module(name=ld, module_type=flip_flop, inputs=[Module(name=gj, module_type=conjunction, inputs=[...], outputs=['ks', 'ld', 'sg', 'xr']), Module(name=sz, module_type=flip_flop, inputs=[Module(name=tr, module_type=flip_flop, inputs=[Module(name=nd, module_type=flip_flop, inputs=[Module(name=ks, module_type=flip_flop, inputs=[Module(name=sc, module_type=flip_flop, inputs=[Module(name=xr, module_type=flip_flop, inputs=[Module(name=gj, module_type=conjunction, inputs=[...], outputs=['ks', 'ld', 'sg', 'xr']), Module(name=broadcaster, module_type=broadcaster, inputs=[], outputs=['pl', 'xr', 'mn', 'xc'])], outputs=['sc', 'gj'])], outputs=['ks', 'gj']), Module(name

In [239]:
modules["jm"].inputs

[Module(name=sg, module_type=conjunction, inputs=[Module(name=gj, module_type=conjunction, inputs=[Module(name=sf, module_type=flip_flop, inputs=[Module(name=qc, module_type=flip_flop, inputs=[Module(name=jf, module_type=flip_flop, inputs=[Module(name=cl, module_type=flip_flop, inputs=[Module(name=ld, module_type=flip_flop, inputs=[Module(name=gj, module_type=conjunction, inputs=[...], outputs=['ks', 'ld', 'sg', 'xr']), Module(name=sz, module_type=flip_flop, inputs=[Module(name=tr, module_type=flip_flop, inputs=[Module(name=nd, module_type=flip_flop, inputs=[Module(name=ks, module_type=flip_flop, inputs=[Module(name=sc, module_type=flip_flop, inputs=[Module(name=xr, module_type=flip_flop, inputs=[Module(name=gj, module_type=conjunction, inputs=[...], outputs=['ks', 'ld', 'sg', 'xr']), Module(name=broadcaster, module_type=broadcaster, inputs=[], outputs=['pl', 'xr', 'mn', 'xc'])], outputs=['sc', 'gj'])], outputs=['ks', 'gj']), Module(name=gj, module_type=conjunction, inputs=[...], outpu

In [351]:
def press_button(modules, n, debug=False):
    MODULE_TO_FIND = "db"
    messages = [("button", "broadcaster", False)]
    while len(messages) > 0:
        msg_from, msg_to, msg = messages.pop(0)
        if msg_from == MODULE_TO_FIND and msg_to == "jm" and msg:
            print(f"{MODULE_TO_FIND} -high jm at {n}!!")
        modules[msg_to].handle_message((msg_from, msg_to, msg), messages, debug)

    return modules

In [352]:
for n in range(1, 100_000):
    modules = press_button(modules, n, debug=False)

db -high jm at 3079!!
db -high jm at 7158!!
db -high jm at 11237!!
db -high jm at 15316!!
db -high jm at 19395!!
db -high jm at 23474!!
db -high jm at 27553!!
db -high jm at 31632!!
db -high jm at 35711!!
db -high jm at 39790!!
db -high jm at 43869!!
db -high jm at 47948!!
db -high jm at 52027!!
db -high jm at 56106!!
db -high jm at 60185!!
db -high jm at 64264!!
db -high jm at 68343!!
db -high jm at 72422!!
db -high jm at 76501!!
db -high jm at 80580!!
db -high jm at 84659!!
db -high jm at 88738!!
db -high jm at 92817!!
db -high jm at 96896!!


In [354]:
import math
# sg =>
# [4027*i-1000 for i in range(1, 10)]
# lm =>
# [3851*i-1000 for i in range(1, 10)]
# dh =>
# [3889*i-1000 for i in range(1, 10)]
# db =>
# [4079*i-1000 for i in range(1, 10)]
math.lcm(4027, 3851, 3889, 4079)

246006621493687